## GPT-3.5を使用したMoodleクイズ問題の生成（トークン数が多めバージョン）

以下のセルのコードを上から順番に実行します。出力になにかおかしなところがあれば、そのセルを再度実行（=regenerate）し、その後のセルも再度実行してください。

Moodleの問題バンクにインポートできるMoodle XMLファイルが作成されます。

In [1]:
# openai python ライブラリーをインストールしていない場合はこれを実行してください。
! pip install openai

In [2]:
data1 = input("何についての問題を生成したい？（最後にエンターキーを）")

何についての問題を生成したい？（最後にエンターキーを） 明治時代の日本史


In [3]:
import openai
openai.api_key = input("openai API key:")

In [4]:
# gpt-3.5-turbo だと、入力＋出力で4096トークン（日本語で4000文字〜5000文字）まで
# 以下のようなMoodle小テスト問題生成だと４問くらいが限界
# model ="gpt-3.5-turbo-0613"
# model ="gpt-3.5-turbo"
# max_tokens=2000

# gpt-3.5-turbo-16k だと、入力＋出力で16000トークン（日本語で1.6万文字くらい）まで
# 利用料は２倍高い https://openai.com/pricing
model="gpt-3.5-turbo-16k"
max_tokens=8000

openai.ChatCompletion.createメソッドのパラメータ：
- model (必須): 使用するモデルのIDを指定します。例えば、"gpt-3.5-turbo"です。
- messages (必須): チャット対話のリストを含むオブジェクトです。各メッセージはrole（"system"、"user"、または"assistant"）とcontent（メッセージのテキスト）を含む辞書として表されます。対話の開始時には、システムメッセージが推奨されます。
- max_tokens (オプション): 応答の最大トークン数を制限します。
- temperature (オプション): モデルの応答の多様性を制御します。値が低いほど、より決定論的な応答が生成されます（0.0に近づくほど決定的になります）、高いほどランダムな応答が生成されます（1.0に近づくほどランダムになります）。
- n (オプション): 応答の候補として生成するトークンの数を制御します。値を増やすと、より多くの応答が生成されますが、処理時間が増える可能性があります。
- stop (オプション): モデルの生成を停止するためのトリガーテキストを指定します。
- logprobs (オプション): 応答のトークンごとの対数確率を返すかどうかを制御します。

In [5]:
num = 8   # 生成する問題の個数
prstr1 = f'''
についての選択問題を{num}個作成してください。以下の条件を満たしてください。
・選択肢は5個
・問題文は丁寧な日本語（ですます調）で書かれている
・選択肢は丁寧ではない日本語（体言止めなど）や単語だけで書かれている
・どの選択肢が正解であるかが示されている
'''[1:-1]

prstr1 += data1
response = openai.ChatCompletion.create(
    model = model,
    max_tokens = max_tokens,
    messages=[
        {"role": "user", "content": prstr1},
    ],
)
res1 = response.choices[0]["message"]["content"].strip()
print(res1)

1. 明治時代の日本の主権は誰にありましたか？
   a. 皇室
   b. 海軍
   c. 貴族
   d. 芸者
   e. 幕府

2. 明治時代の日本で最初に採用された憲法は何と呼ばれていましたか？
   a. 大日本帝国憲法
   b. 明治憲法
   c. 幕藩体制
   d. 王政復古
   e. 戦国時代

3. 明治時代の日本で、西洋式の近代産業を導入するために作られた政府機関は何ですか？
   a. 明治天皇
   b. アメリカ合衆国
   c. 日本銀行
   d. 参議府
   e. 政府の役人

4. 明治時代の日本で、脱藩して新しい町を作る動きは何と呼ばれていましたか？
   a. 土地社会主義
   b. 日本列島
   c. 芸者文化
   d. 世襲制度
   e. 小泉純一郎

5. 明治時代の日本で、外国の文化が取り入れられた結果、新しい流行が生まれました。この流行は何と呼ばれていましたか？
   a. 和製英語
   b. 明治維新
   c. 踊り忘れ
   d. 万博
   e. 西洋狂

6. 明治時代の日本で最初に国民が選挙で議員を選ぶことができるようになったのはいつですか？
   a. 1945年
   b. 1873年
   c. 1614年
   d. 1912年
   e. 1867年

7. 明治時代の日本で、教育が一般化するために何が創設されましたか？
   a. 高等師範学校
   b. ドクターストップ
   c. 系統図
   d. 子供手当
   e. 幼稚園

8. 明治時代の日本で、戦国時代のような戦乱が起きないようにするために、何が廃止されましたか？
   a. 武士道
   b. 和製英語
   c. 芸者文化
   d. アメリカ合衆国
   e. 荘園制度


In [7]:
prstr2 = '''
今のすべての問題を一連のデータとして（問題ごとに分けずに） 以下のような Moodle XML 形式で出力してください。選択肢の句点は削除してください。

  <question type="multichoice">
    <name>
      <text>問いの要約</text>
    </name>
    <questiontext format="html">
      <text><![CDATA[問いの文章をここに書く。]]></text>
    </questiontext>
    <shuffleanswers>true</shuffleanswers>
    <answer fraction="0">
      <text>不正解</text>
    </answer>
    <answer fraction="100">
      <text>正解</text>
    </answer>
    <answer fraction="0">
      <text>不正解</text>
    </answer>
    <answer fraction="0">
      <text>不正解</text>
    </answer>
    <generalfeedback>
      <text><![CDATA[全般に対するフィードバック]]></text>
    </generalfeedback>
  </question>
'''[1:]

response = openai.ChatCompletion.create(
    model = model,
    max_tokens = max_tokens,
    messages=[
        {"role": "user", "content": prstr1},
        {"role": "assistant", "content": res1},
        {"role": "user", "content": prstr2},
    ],
)

res2 = response.choices[0]["message"]["content"].strip()
print(res2)

<quiz>
  <question type="multichoice">
    <name>
      <text>問題1</text>
    </name>
    <questiontext format="html">
      <text><![CDATA[明治時代の日本の主権は誰にありましたか？]]></text>
    </questiontext>
    <shuffleanswers>true</shuffleanswers>
    <answer fraction="0">
      <text>皇室</text>
    </answer>
    <answer fraction="0">
      <text>海軍</text>
    </answer>
    <answer fraction="0">
      <text>貴族</text>
    </answer>
    <answer fraction="0">
      <text>芸者</text>
    </answer>
    <answer fraction="100">
      <text>幕府</text>
    </answer>
    <generalfeedback>
      <text><![CDATA[明治時代の主権は江戸幕府にありました。]]></text>
    </generalfeedback>
  </question>
  <question type="multichoice">
    <name>
      <text>問題2</text>
    </name>
    <questiontext format="html">
      <text><![CDATA[明治時代の日本で最初に採用された憲法は何と呼ばれていましたか？]]></text>
    </questiontext>
    <shuffleanswers>true</shuffleanswers>
    <answer fraction="100">
      <text>大日本帝国憲法</text>
    </answer>
    <answer fraction="0">
      <text>明治憲

In [8]:
prstr3 = '''
以下のように直してください。
・<quiz> から開始する
・選択肢番号があれば削除
・正解の選択肢の配点は 100 になっている
・問題名（<name><text> と </text></name> の間）は、問いの文章を10文字程度で要約したもの
・各問題に対して1つだけ、</questiontext> の後に、200文字程度の「全般的なフィードバック」を <generalfeedback><text> と </text></generalfeedback> で囲って書く
・「全般的なフィードバック」には、参考となるWikipediaページなどのリンクが target=_BLANK の指定付きで含まれている
・各問題に対して <shuffleanswers>true</shuffleanswers> を指定する
・[CDATA[ に対応する ]] がちゃんと書かれている
'''[1:]

response = openai.ChatCompletion.create(
    model = model,
    max_tokens = max_tokens,
    messages=[
        {"role": "user", "content": prstr2},
        {"role": "assistant", "content": res2},
        {"role": "user", "content": prstr3},
    ],
)
res3 = response.choices[0]["message"]["content"].strip()
print(res3)

with open('moodle-quiz-questions01.xml', 'w') as f:
    print(res3, file=f)

<quiz>
  <question type="multichoice">
    <name>
      <text>明治時代の主権</text>
    </name>
    <questiontext format="html">
      <text><![CDATA[明治時代の日本の主権は誰にありましたか？]]></text>
    </questiontext>
    <shuffleanswers>true</shuffleanswers>
    <answer fraction="0">
      <text>皇室</text>
    </answer>
    <answer fraction="0">
      <text>海軍</text>
    </answer>
    <answer fraction="0">
      <text>貴族</text>
    </answer>
    <answer fraction="100">
      <text>幕府</text>
    </answer>
    <generalfeedback>
      <text><![CDATA[明治時代の主権は<a href="https://ja.wikipedia.org/wiki/%E6%B1%9F%E6%88%B8%E5%B9%95%E5%BA%9C_(%E6%98%8E%E6%B2%BB)" target="_BLANK">江戸幕府</a>にありました。]]></text>
    </generalfeedback>
  </question>
  <question type="multichoice">
    <name>
      <text>明治時代の憲法</text>
    </name>
    <questiontext format="html">
      <text><![CDATA[明治時代の日本で最初に採用された憲法は何と呼ばれていましたか？]]></text>
    </questiontext>
    <shuffleanswers>true</shuffleanswers>
    <answer fraction="0">
      <text>明治憲法</te